# 3. Máquinas de Vectores de Soporte <a id="10"></a>

## Leer el Conjunto de Datos

Cargar los datos y guardarlos en el dataframe `df5`:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
# ruta de datos y leer los datos

path5='datos/cell_samples.csv'
df5 = pd.read_csv(path5)
df5.head()

Tamaño y forma del conjunto de datos.

In [ ]:
print('Tamaño: ', df5.size)
print('Forma: ', df5.shape)

El campo `ID` contiene los identificadores de los pacientes. Las características de las muestras de células de cada paciente están contenidas entre los campos `Clump` a `Mit`. Los valores se clasifican del 1 al 10, siendo 1 el más cercano a benigno.

El campo Clase contiene el diagnóstico, confirmado por procedimientos médicos separados, en cuanto a si las muestras son benignas (valor = 2) o malignas (valor = 4).

Por ejemplo, se puede observar la distribución de las clases según el grosor de los grumos y la uniformidad del tamaño de la celda para los 50 primeros registros.

In [ ]:
ax = df5[df5['Class'] == 4][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='DarkBlue', label='maligno');
df5[df5['Class'] == 2][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='Yellow', label='benigno', ax=ax);
plt.show()

## Pre-procesamiento de los Datos

Lo primero es observar los tipos de datos de las columnas del conjunto de datos.

In [ ]:
df5.dtypes

Aquí se puede ver que la columna `BareNuc` incluye algunos valores que no son numéricos. Por este motivo se va a proceder a eliminar esas filas. Para hacer esto se usará el método de *pandas* `.to_numeric` con el parámetro `error=coerce` para convertir los valores inválidos como `NaN`. A su vez, se le aplicará el método `.notnull()` para eliminar las filas que contengan un valor `NaN`. Se terminará esta etapa convirtiendo los valores al tipo entero.

In [ ]:
df5 = df5[pd.to_numeric(df5['BareNuc'], errors='coerce').notnull()]
df5['BareNuc'] = df5['BareNuc'].astype('int')
df5.dtypes

Con el atributo `shape` se puede observar que se eliminaron 16 filas, es decir se pasó de 699 filas a 683 filas.

In [ ]:
print('Forma: ', df5.shape)

Ahora, ya está todo listo para definir **X5** e **y5** a partir del dataframe ya procesado.

In [ ]:
X5 = np.asarray(df5[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']])
X5[0:5]

Recordar que el objetivo es crear un modelo que prediga el valor de la clase, es decir, benigno (=2) o maligno (=4). 

In [ ]:
y5 = np.asarray(df5['Class'])
y5 [0:5]

## Configuración del Modelo

El siguiente paso es dividir el conjunto de datos en el conjunto de entrenamiento y el conjunto de prueba

In [ ]:
X5_entrena, X5_prueba, y5_entrena, y5_prueba = train_test_split(X5, y5, test_size=0.2, random_state=4)
print ('Conjunto de Entrenamiento set:', X5_entrena.shape,  y5_entrena.shape)
print ('Conjunto de Prueba:', X5_prueba.shape,  y5_prueba.shape)

## Modelado

El algoritmo SVM ofrece una colección de funciones de **kernel** para realizar su procesamiento. Básicamente, el mapeo de datos en un espacio dimensional superior se llama **kernelling**. La función matemática utilizada para la transformación se conoce como **función kernel** y puede ser de diferentes tipos, como por ejemplo:

1. Lineal.
2. Polinomial
3. Función de base radial (RBF).
4. Sigmoide.

Cada una de estas funciones tiene sus características, sus pros y sus contras, sin embargo, no hay una manera fácil de saber qué función funcionará mejor con un conjunto de datos determinado. Generalmente, se eligen diferentes funciones a la vez y se comparan los resultados. En esta oportunidad se usará el kernel RBF (función de base radial) para el desarrollo de esta parte.

Ahora se procederá a importar el módulo `svm` y posteriormente se ajustará el modelo con el conjunto de entrenamiento.

In [ ]:
from sklearn import svm

modelo_svm = svm.SVC(kernel='rbf')
modelo_svm.fit(X5_entrena, y5_entrena) 

## Pronóstico

Una vez entrenado el modelo se puede realizar el pronóstico con el conjunto de datos de prueba.

In [ ]:
y5_hat = modelo_svm.predict(X5_prueba)
y5_hat

## Evaluación

Para el paso de evaluación se va a reutilizar la función `grafica_matrix_confusion` para observar la precisión del modelo generado.

In [ ]:
# Esta función imprime y grafica una matriz de confusión.
# Se puede aplicar una normalización configurando el parámetro `normalize=True`.

import itertools

def grafica_matriz_confusion(matr_conf, clases,
                          normalizar=False,
                          titulo='Matriz de Confusión',
                          cmap=plt.cm.Blues):

    if normalizar:
        matr_conf = matr_conf.astype('float') / matr_conf.sum(axis=1)[:, np.newaxis]
        print("Matriz de Confusión Normalizada.")
    else:
        print('Matriz de Confusión matrix sin normalización')

    print(matr_conf)

    plt.imshow(matr_conf, interpolation='nearest', cmap=cmap)
    plt.title(titulo)
    plt.colorbar()
    tick_marks = np.arange(len(clases))
    plt.xticks(tick_marks, clases, rotation=45)
    plt.yticks(tick_marks, clases)

    formato = '.2f' if normalizar else 'd'
    umbral = matr_conf.max() / 2.
    for i, j in itertools.product(range(matr_conf.shape[0]), range(matr_conf.shape[1])):
        plt.text(j, i, format(matr_conf[i, j], formato),
                 horizontalalignment="center",
                 color="white" if matr_conf[i, j] > umbral else "black")

    plt.tight_layout()
    plt.ylabel('Etiqueta valores Verdaderos')
    plt.xlabel('Etiqueta valores Pronosticados')

In [ ]:
# Calcular matriz de confusión
matriz_confusion = metrics.confusion_matrix(y5_prueba, y5_hat, labels=[2,4])
np.set_printoptions(precision=2)

print (metrics.classification_report(y5_prueba, y5_hat))

# Grafica matriz de confusión no normalizada
plt.figure()
grafica_matriz_confusion(matriz_confusion, clases=['Benigno(2)','Maligno(4)'],normalizar= False,  titulo='Matriz de Confusión')



También se puede usar fácilmente el **Valor de F1** de la biblioteca `sklearn`.


In [ ]:
metrics.f1_score(y5_prueba, y5_hat, average='weighted') 

Por otro lado, además se puede probar el **índice de Jaccard** para medir la precisión.

In [ ]:
metrics.jaccard_score(y5_prueba, y5_hat, pos_label=2)